In [4]:
!pip install ipython
!pip3 install ipython


In [5]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json
from IPython.display import Image
from IPython.display import display

api_key = 'ab4a7f41157e4592aa7cf47de4be49a1'
host = 'westcentralus.api.cognitive.microsoft.com'
personGroupId = 'jiangsir_groupid4'

def identify(faceids, personGroupId):
    print("開始辨識。")
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': api_key,
    }

    params = urllib.parse.urlencode({
    })

    requestbody = '''{
        "personGroupId": "'''+personGroupId+'''",
        "faceIds":'''+str(faceids)+''',
        "maxNumOfCandidatesReturned":1,
        "confidenceThreshold": 0.5
    }'''

    try:
        conn = http.client.HTTPSConnection(host)
        conn.request("POST", "/face/v1.0/identify?%s" % params, requestbody, headers)
        response = conn.getresponse()
        data = response.read()
        #print(data)
        facejson = json.loads(str(data,'UTF-8'))
        #print(facejson)
        conn.close()
        return facejson
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

        
def get_a_person(personGroupId, personId):
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': api_key,
    }

    params = urllib.parse.urlencode({
    })

    try:
        conn = http.client.HTTPSConnection(host)
        conn.request("GET", "/face/v1.0/persongroups/"+personGroupId+"/persons/"+personId+"?%s" % params, "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        personjson = json.loads(str(data,'UTF-8'))
        conn.close()
        return personjson
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
        

def list_persons_in_group(personGroupId):
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': api_key,
    }

    params = urllib.parse.urlencode({
        # Request parameters
        #'start': '{string}',
        #'top': '1000',
    })

    try:
        conn = http.client.HTTPSConnection(host)
        conn.request("GET", "/face/v1.0/persongroups/"+personGroupId+"/persons?%s" % params, "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        #print(data)
        persons = json.loads(str(data,'UTF-8'))
        conn.close()
        return persons
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))    

def DetectingLocal(imagepath):
    headers = {
        # Request headers
        #'Content-Type': 'application/json',
        'Content-Type': 'application/octet-stream', # 用本地圖檔辨識
        'Ocp-Apim-Subscription-Key': api_key,
    }

    params = urllib.parse.urlencode({
        # Request parameters
        'returnFaceId': 'true',
        'returnFaceLandmarks': 'false',
        'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure',
    })

    #requestbody = '{"url":"'+imageurl+'"}'
    #requestbody = open('face1.JPG', "rb").read()
    requestbody = open(imagepath, "rb").read()
    
    try:
        conn = http.client.HTTPSConnection(host)
        conn.request("POST", "/face/v1.0/detect?%s" % params, requestbody, headers)
        response = conn.getresponse()
        data = response.read()
        #print(data)
        parsed = json.loads(str(data,'UTF-8'))
        print(parsed)
        #print(parsed[0]['faceId'])
        #faceids.append(parsed[0]['faceId'])
        conn.close()
        return parsed[0]['faceId']
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))


##################################################################################################
### main
##################################################################################################

faceids = []
imagepaths = []
imagepaths.append("face1.JPG")
imagepaths.append("face2.jpg")
#imagepaths.append("face3.jpg")
imagepaths.append("face4.jpg")

#for imageurl in imageurls:
#    faceids.append(Detecting(imageurl))
for imagepath in imagepaths:
    faceids.append(DetectingLocal(imagepath))



persons = list_persons_in_group(personGroupId)    
for person in persons:
    print(person["name"], person["personId"])

facejson = identify(faceids, personGroupId)
print(facejson)
for i in range(len(faceids)):
    if len(facejson[i]["candidates"])>0:
        print("personId: "+facejson[i]["candidates"][0]["personId"]+", 信心指數："+str(facejson[i]["candidates"][0]["confidence"]))
        personjson = get_a_person(personGroupId, facejson[i]["candidates"][0]["personId"])
        print("影中人："+personjson['name'])
    else:
        print("沒有符合的人！")
    display(Image(url=imagepaths[i]))


[{'faceRectangle': {'top': 141, 'height': 261, 'width': 261, 'left': 664}, 'faceId': 'fea57809-c7aa-4a7f-a5b0-a54861e23b76', 'faceAttributes': {'blur': {'value': 0.0, 'blurLevel': 'low'}, 'makeup': {'lipMakeup': False, 'eyeMakeup': False}, 'smile': 0.999, 'exposure': {'exposureLevel': 'goodExposure', 'value': 0.59}, 'occlusion': {'foreheadOccluded': False, 'eyeOccluded': False, 'mouthOccluded': False}, 'emotion': {'sadness': 0.0, 'fear': 0.0, 'anger': 0.0, 'disgust': 0.0, 'happiness': 0.999, 'surprise': 0.0, 'neutral': 0.001, 'contempt': 0.0}, 'headPose': {'yaw': -14.1, 'pitch': 0.0, 'roll': -13.9}, 'hair': {'bald': 0.01, 'hairColor': [{'confidence': 1.0, 'color': 'black'}, {'confidence': 0.38, 'color': 'other'}, {'confidence': 0.24, 'color': 'gray'}, {'confidence': 0.2, 'color': 'red'}, {'confidence': 0.2, 'color': 'brown'}, {'confidence': 0.04, 'color': 'blond'}], 'invisible': False}, 'glasses': 'ReadingGlasses', 'accessories': [{'confidence': 1.0, 'type': 'glasses'}], 'facialHair': 

沒有符合的人！


沒有符合的人！
